<a href="https://colab.research.google.com/github/PaulB86UK/Kaggle-Jane_Street/blob/main/Jane_Enero_2021-Autoencoder_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
#Loading Data
!pip install datatable
!pip install -U keras-tuner
import datatable as dt

#Preprocessing & EDA
import pandas as pd
import numpy as np

#Visualizations
import seaborn as sns
import matplotlib.pyplot as plt

#Modelling
import tensorflow as tf
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
#import xgboost as xgb
#from sklearn.metrics import roc_auc_score, roc_curve
#from sklearn.model_selection import GroupKFold

#Hyperparameter Tunning
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
import kerastuner as kt
#OS
import warnings
warnings.filterwarnings('ignore')
import os, gc #garbage collection

#Seeding
tf.random.set_seed(42)

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.2)


In [24]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]


class CVTuner(kt.engine.tuner.Tuner):
    def run_trial(self, trial, X, y, splits, batch_size=32, epochs=1,callbacks=None):
        val_losses = []
        for train_indices, test_indices in splits:
            X_train, X_test = [x[train_indices] for x in X], [x[test_indices] for x in X]
            y_train, y_test = [a[train_indices] for a in y], [a[test_indices] for a in y]
            if len(X_train) < 2:
                X_train = X_train[0]
                X_test = X_test[0]
            if len(y_train) < 2:
                y_train = y_train[0]
                y_test = y_test[0]
            
            model = self.hypermodel.build(trial.hyperparameters)
            hist = model.fit(X_train,y_train,
                      validation_data=(X_test,y_test),
                      epochs=epochs,
                        batch_size=batch_size,
                      callbacks=callbacks)
            
            val_losses.append([hist.history[k][-1] for k in hist.history])
        val_losses = np.asarray(val_losses)
        self.oracle.update_trial(trial.trial_id, {k:np.mean(val_losses[:,i]) for i,k in enumerate(hist.history.keys())})
        self.save_model(trial.trial_id, model)

In [6]:
%%time
#COLAB
!unzip /content/drive/MyDrive/ORDENAR/JaneStreet/train.csv.zip 
train_dt = dt.fread("/content/train.csv")
train = train_dt.to_pandas()
features = pd.read_csv("/content/drive/MyDrive/ORDENAR/JaneStreet/features.csv")
examples_test = pd.read_csv("/content/drive/MyDrive/ORDENAR/JaneStreet/example_test.csv")
example_sample_submission = pd.read_csv("/content/drive/MyDrive/ORDENAR/JaneStreet/example_sample_submission.csv")
#KAGGLE
'''train_dt = dt.fread("../input/jane-street-market-prediction/train.csv")
train = train_dt.to_pandas()
train = train.astype({c: np.float16 for c in train.select_dtypes(include='float64').columns}) #limit memory use from 64 to 16 was 32
features = pd.read_csv("../input/jane-street-market-prediction/features.csv")
examples_test = pd.read_csv("../input/jane-street-market-prediction/example_test.csv")
example_sample_submission = pd.read_csv("../input/jane-street-market-prediction/example_sample_submission.csv")'''

Archive:  /content/drive/MyDrive/ORDENAR/JaneStreet/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
CPU times: user 36.3 s, sys: 7.51 s, total: 43.9 s
Wall time: 3min 54s


In [11]:
%%time
del train_dt
gc.collect
train = train.interpolate(method='polynomial', order=2)
features = [c for c in train.columns if 'feature' in c]
f_mean = train[features[1:]].mean()
train = train.query('weight > 0').reset_index(drop = True)
#train['action'] = (train['resp_4'] > 0.0001).astype('int') nono 21
train = train[train.date > 85].reset_index(drop = True)
corr_resp_4 = train[features].corrwith(train.resp_4)
print(corr_resp_4[abs(corr_resp_4) >= 0.01].shape) # 26 values 
corr_resp_4 = corr_resp_4[abs(corr_resp_4) > 0.001]
columnas_resp4 = list(corr_resp_4.index)
columnas_resp4
corr_weight = train[features].corrwith(train.weight)
print(corr_weight[abs(corr_weight) >= 0.01].shape) #62
corr_weight = corr_weight[abs(corr_weight) > 0.01]
columnas_weight = list(corr_weight.index)
columnas_weight
columnas_total =  columnas_weight + columnas_resp4
columnas_total =  pd.unique(columnas_total).tolist()
print(len(columnas_total)) 
label_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
X = train[columnas_total].values
y = np.stack([(train[c] > 0.000001).astype('int') for c in label_cols]).T

(26,)
(75,)
117
CPU times: user 56.7 s, sys: 11.8 s, total: 1min 8s
Wall time: 1min 8s


In [ ]:
TRAINING  = True
def create_autoencoder(hp, input_dim,output_dim,noise):
    aenc_out = hp.Int('aenc_out',128,256)
    aenc_in = hp.Int('aenc_in',32,96)
    aenc_d_out = hp.Float('aenc_d_out',0.15,0.35)
    aenc_d_out_2 = hp.Float('aenc_d_out_2',0.05,0.25)
    inputs = Input(input_dim)
    encoded = BatchNormalization()(inputs)
    encoded = GaussianNoise(noise)(encoded)
    encoded = Dense(aenc_out,activation='relu')(encoded) 
    encoded = BatchNormalization()(encoded) 
    encoded = tf.keras.layers.Dropout(aenc_d_out)(encoded) 
    encoded = Dense(aenc_in,activation='relu')(encoded) # activity_regularizer=regularizers.l1(10e-5)
    encoded = BatchNormalization()(encoded)
    decoded = Dropout(aenc_d_out)(encoded)
    decoded = Dense(aenc_in,activation='relu')(decoded)
    decoded = BatchNormalization()(decoded)
    decoded = tf.keras.layers.Dropout(aenc_d_out)(decoded)
    decoded = Dense(input_dim,name='decoded')(decoded)
    x = Dense(aenc_out*2/3,activation='relu')(decoded)
    x = BatchNormalization()(x)
    x = Dropout(aenc_d_out_2)(x)
    x = Dense(aenc_out*1/3,activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(aenc_d_out_2)(x)
    x = Dense(output_dim,activation='sigmoid',name='label_output')(x)
    
    model = Model(inputs=inputs,outputs=x)
    model.compile(optimizer=Adam(hp.Float('lr',0.00001,0.1,default=0.001)),loss=BinaryCrossentropy(label_smoothing=hp.Float('label_smoothing',0.0,0.1)),metrics=[tf.keras.metrics.AUC(name = 'auc')])
    return model

model_fn = lambda hp: create_autoencoder(hp,X.shape[-1],y.shape[-1],0.1)

tuner = CVTuner(
        hypermodel=model_fn,
        oracle=kt.oracles.BayesianOptimization(
        objective= kt.Objective('val_auc', direction='max'),
        num_initial_points=4,
        max_trials=100))

FOLDS = 3
SEED = 42

#add all the seeding!!!! move to tensorflow with tpu at some point!!
tf.random.set_seed(SEED)

if TRAINING:
    gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20)
    splits = list(gkf.split(y, groups=train['date'].values))
    tuner.search((X,),(y,),splits=splits,batch_size=16384,epochs=300,callbacks=[EarlyStopping('val_auc', mode='max',patience=3)])
    hp  = tuner.get_best_hyperparameters(1)[0]
    pd.to_pickle(hp,f'./best_hp_{SEED}.pkl')
    for fold, (train_indices, test_indices) in enumerate(splits):
        model = model_fn(hp)
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
        model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=16384,callbacks=[EarlyStopping('val_auc',mode='max',patience=10,restore_best_weights=True)])
        model.save_weights(f'./model_{SEED}_{fold}.hdf5')
        model.compile(Adam(hp.get('lr')/100),loss='binary_crossentropy')
        model.fit(X_test,y_test,epochs=6,batch_size=16384)
        model.save_weights(f'./model_{SEED}_{fold}_finetune.hdf5')
    tuner.results_summary()



Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
aenc_out          |147               |?                 
aenc_in           |80                |?                 
aenc_d_out        |0.24643           |?                 
aenc_d_out_2      |0.15836           |?                 
lr                |0.070478          |?                 
label_smoothing   |0.068458          |?                 

Epoch 1/300
17/17 [==============================] - 17s 915ms/step - loss: 0.7443 - auc: 0.5016 - val_loss: 34.9246 - val_auc: 0.4990
Epoch 2/300
17/17 [==============================] - 14s 820ms/step - loss: 0.6953 - auc: 0.5038 - val_loss: 0.8268 - val_auc: 0.5079
Epoch 3/300
17/17 [==============================] - 14s 827ms/step - loss: 0.6937 - auc: 0.5073 - val_loss: 0.7288 - val_auc: 0.5152
Epoch 4/300
17/17 [==============================] - 14s 829ms/step - loss: 0.6928 - auc: 0.5168 - val_loss: 0.7326 - val_auc: 0.5252
Epoch 5/300
17/17 [=================